In [4]:
import xarray as xr
import os
import xarray as xr
import numpy as np
import metpy
import metpy.calc as mpcalc
from metpy.units import units
from datetime import datetime
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.patches as mpatches
import glob
import pandas as pd
from PIL import Image
from olr_to_tb import *
import math

In [ ]:
datasets = [xr.open_dataset('mcs/' + f) for f in os.listdir('mcs')]
datasets = [ds.swap_dims({'Time': 'XTIME'}) for ds in datasets]
combined_dataset = xr.concat(datasets, dim='XTIME')

In [ ]:
combined_dataset = combined_dataset.rename_dims({'XTIME': 'time'})

In [ ]:
time0 = combined_dataset['time'][0]
for coord in combined_dataset.coords:
    if coord != 'XTIME':
        combined_dataset[coord] = combined_dataset[coord].sel(time = time0)

combined_dataset

In [ ]:
# Create temperature profile
p = combined_dataset['PB'] + combined_dataset['P']  # Base pressure + Perturbation pressure

# Compute total potential temperature
theta = combined_dataset['T'] + combined_dataset['T00']  # Perturbation potential temperature + base state offset

# Reference pressure
p0 = combined_dataset['P00'] 

# Constants
R = 287.05    # J/(kg·K)
cp = 1004.0   # J/(kg·K)

# Compute temperature
T = theta * (p / p0)**(R / cp)

# Add to dataset for convenience
combined_dataset['T_actual'] = T

In [ ]:
# create shear and cape variables (IN PROGRESS)
temperature = combined_dataset['T_actual'] * units.kelvin
mixing_ratio = combined_dataset['QVAPOR']

vapor_pressure = mpcalc.vapor_pressure(p * units.Pa, mixing_ratio * units.kg / units.kg)
dewpoint = mpcalc.dewpoint(vapor_pressure)

capes = np.zeros_like(combined_dataset['T2'])

pressures = p * units.Pa
# Calculate CAPE (requires pressure and temperature)
for i in range(len(combined_dataset['XTIME'])):
    print(i)
    for j in range(len(combined_dataset['south_north'])):
        for k in range(len(combined_dataset['west_east'])):
            #capes[i, j, k] = 1 # to test size capacity
            capes[i, j, k] = mpcalc.surface_based_cape_cin(pressures[i, :, j, k], temperature[i, :, j, k], dewpoint[i, :, j, k])[0].magnitude


# Assign CAPE as a new variable in the dataset
#

In [ ]:
combined_dataset['CAPE'] = (['time', 'south_north', 'west_east'], capes)

In [56]:
combined_dataset['U10K'] = combined_dataset['U10'] * 1.94384
combined_dataset['V10K'] = combined_dataset['V10'] * 1.94384
combined_dataset['UK'] = combined_dataset['U'] * 1.94384
combined_dataset['VK'] = combined_dataset['V'] * 1.94384

In [ ]:
g = 9.81  # Acceleration due to gravity (m/s^2)
z = (combined_dataset['PH'] + combined_dataset['PHB']) / g  # Geopotential height in meters
z = 0.5 * (z[:, :-1, :, :] + z[:, 1:, :, :])
z_target = 6000  # Target height in meters

bottom_top_levels = combined_dataset['bottom_top_stag']
z_diff = np.abs(z - z_target)
z_index = z_diff.argmin(dim='bottom_top_stag')  # Index of closest level to 6 km

u_unstaggered = 0.5 * (combined_dataset['UK'][:, :, :, :-1] + combined_dataset['UK'][:, :, :, 1:])
v_unstaggered = 0.5 * (combined_dataset['VK'][:, :, :-1, :] + combined_dataset['VK'][:, :, 1:, :])

u_unstaggered = u_unstaggered.rename({'west_east_stag': 'west_east'})
v_unstaggered = v_unstaggered.rename({'south_north_stag': 'south_north'})

u_6km = u_unstaggered.isel(bottom_top=z_index)
v_6km = v_unstaggered.isel(bottom_top=z_index)

u_0km = combined_dataset['U10K'] 
v_0km = combined_dataset['V10K']

delta_u = u_6km.values - u_0km
delta_v = v_6km.values - v_0km

# Calculate wind shear magnitude
shear = np.sqrt(np.square(delta_u) + np.square(delta_v))

combined_dataset['U_shear_6'] = delta_u
combined_dataset['V_shear_6'] = delta_v
combined_dataset['mag_shear_6'] = shear

In [ ]:
g = 9.81  # Acceleration due to gravity (m/s^2)
z = (combined_dataset['PH'] + combined_dataset['PHB']) / g  # Geopotential height in meters
z = 0.5 * (z[:, :-1, :, :] + z[:, 1:, :, :])
z = z.rename({'bottom_top_stag': 'bottom_top'})

In [ ]:
brightness_temp = olr_to_tb(combined_dataset['OLR'])
combined_dataset['TB'] = brightness_temp
temp_difs = np.abs(T - brightness_temp)
temp_index = temp_difs.argmin(dim='bottom_top')
cloud_heights = z.isel(bottom_top = temp_index)
combined_dataset['CL_HT'] = cloud_heights

In [12]:
combined_dataset['T2F'] = (combined_dataset['T2'] - 273.15) * 9/5 + 32
combined_dataset['RAIN'] = combined_dataset['RAINC'] + combined_dataset['RAINNC']

In [5]:
# if we've already created the dataset
combined_dataset = xr.open_dataset('excluded_data/dataset.nc')

In [57]:
combined_dataset['U_1'] = combined_dataset['UK'].isel(bottom_top = 0)
combined_dataset['V_1'] = combined_dataset['VK'].isel(bottom_top = 0)
combined_dataset['WIND_sfc_mag'] = np.sqrt(np.square(combined_dataset['U10K']) + np.square(combined_dataset['V10K']))
combined_dataset['theta'] = combined_dataset['T'] + combined_dataset['T00']

In [53]:
rains = np.zeros_like(combined_dataset['RAIN'])
tot_rain = combined_dataset['RAINC'] + combined_dataset['RAINNC']
rains[0, :, :] = np.zeros_like(combined_dataset['RAIN'].isel(time = 0))
rains[12, :, :] = np.zeros_like(combined_dataset['RAIN'].isel(time = 0))
for i in range(1, 12):
    rains[i, :, :] = tot_rain.isel(time = i) - tot_rain.isel(time = i - 1)
for i in range(13, 25):
    rains[i, :, :] = tot_rain.isel(time = i) - tot_rain.isel(time = i - 1)
combined_dataset['RAIN'] = (['time', 'south_north', 'west_east'], rains)

In [55]:
def make_maps(ds, var, title, x_barb = None, y_barb = None, cmap = 'viridis', bottom = None, top = None, bottom_top = 0, show = False, save = False, save_loc = None):
    if cmap == 'grey_to_white':
        cmap = plt.cm.gray  # Base colormap
        gray_to_white = cmap(np.linspace(0.5, 1, 256))  # Use the upper half of the colormap
        cmap = plt.matplotlib.colors.ListedColormap(gray_to_white)
    
    if 'bottom_top' in ds[var].dims:
        global_min = ds[var].isel(bottom_top=bottom_top).min().values
        global_max = ds[var].isel(bottom_top=bottom_top).max().values
    else:
        global_min = ds[var].min().values
        global_max = ds[var].max().values
    if bottom:
        global_min = bottom
    if top:
        global_max = top


    range_mag = 10 ** math.floor(math.log10(global_max - global_min))

    # Round down min_val and up max_val to the nearest multiple of range_mag
    global_min = math.floor(global_min / range_mag) * range_mag
    if bottom == 2000: # manual override for cloud heights
        global_min = 2000
    global_max = math.ceil(global_max / range_mag) * range_mag

    for i, time in enumerate(ds['XTIME']):
        frame = ds.isel(time = i)[var]
        if x_barb:
            frameu = ds.isel(time = i)[x_barb].thin(10)
            framev = ds.isel(time = i)[y_barb].thin(10)
        if 'bottom_top' in frame.dims:
            frame = frame.sel(bottom_top = bottom_top)
        if x_barb:
            if 'bottom_top' in frameu.dims:
                frameu = frameu.sel(bottom_top = bottom_top)
            if 'bottom_top' in framev.dims:
                framev = framev.sel(bottom_top = bottom_top)
        lat_coord = [name for name in frame.coords if "LAT" in name][0]
        long_coord = [name for name in frame.coords if "LONG" in name][0]

        fig = plt.figure(figsize=(16, 12))
        ax = fig.add_subplot(111, projection=ccrs.LambertConformal())
        ax.set_extent([-130, -107, 39, 51])
        ax.add_feature(cfeature.COASTLINE.with_scale('50m'), linewidth=1)
        ax.add_feature(cfeature.STATES.with_scale('50m'), linewidth=0.5, edgecolor='black')
        ax.add_feature(cfeature.BORDERS.with_scale('50m'), linewidth=1, edgecolor='black')
        
        levels = np.linspace(global_min, global_max, 11)
        plt1 = ax.contourf(
            frame[long_coord], frame[lat_coord], frame, levels = levels, cmap=cmap,
            transform=ccrs.PlateCarree(), vmin=global_min, vmax=global_max)
        if x_barb:
            plt2 = ax.barbs(
                frameu[long_coord].values, frameu[lat_coord].values, frameu.values, framev.values, transform=ccrs.PlateCarree(), length = 6)
        cb = plt.colorbar(plt1, ax=ax, orientation='horizontal', pad=0.05, aspect=30)
        ax.set_title(
            'WRF ' + title,
            fontweight='bold', fontsize=14, loc='left')
        #dt = datetime.utcfromtimestamp(time.values.astype(int) * 1e-9)
        ax.set_title(time.values, fontsize=14, loc='right')
        
        # # Format the gridlines (optional)
        gl = ax.gridlines(
            crs=ccrs.PlateCarree(), draw_labels=True, dms=True, x_inline=False,
            y_inline=False, linewidth=1, color='k', linestyle=':')
        gl.xlocator = mticker.FixedLocator([-130, -125, -120, -115, -110])
        gl.ylocator = mticker.FixedLocator([40, 45, 50, 55])
        gl.top_labels = False
        gl.right_labels = False
        gl.xlabel_style = {'size': 16, 'rotation': 20}
        gl.ylabel_style = {'size': 16}

        print(pd.to_datetime(str(time.values)).strftime('%d%H'))
        if save:
            plt.savefig(save_loc + pd.to_datetime(str(time.values)).strftime('%d%H') + '.png')
        if show:
            plt.show()
        
        plt.clf()

#make_maps(combined_dataset, 'RAIN', 'Test', bottom = .1, top = 10, show = True)

In [59]:
def make_gif(frame_folder):
    frames = [Image.open(image) for image in glob.glob(f"{frame_folder}/*.png")]
    frame_one = frames[0]
    frame_one.save(f"{frame_folder}/loop.gif", format="GIF", append_images=frames,
               save_all=True, duration=300, loop=0)

In [ ]:
# Radar reflectivity (most important), wind, precip, OLR, CAPE (would need to calculate?)
vars_of_interest = ['REFL_10CM', 'OLR', 'T2F', 'mag_shear_6', 'CL_HT', 'TB', 'CAPE', 'RAIN', 'WIND_sfc_mag', 'theta'] # And cape with shear once shear and cape created as variables 
titles = ['Reflectivity (DBZ)', 'Outgoing Longwave Radiation (W/m2)', 
          '2m Temperature (F) and Wind Barbs', '0-6 km Wind Shear (Magnitude in kt and Barbs)', 'Cloud Top Height (m; Derived from Brightness Temp)', 
          'Brightness Temperature (K)', 'CAPE (J/kg) and 0-6 km Wind Shear Barbs', 'Hourly Rain (mm)', 'Surface Wind (kt)', 'Potential Temperature (K) and Level 0 Wind Barbs']

x_barbs = [None, None, 'U10', 'U_shear_6', None, None, 'U_shear_6', None, 'U10', 'U_1']
y_barbs = [None, None,  'V10', 'V_shear_6', None, None, 'V_shear_6', None, 'V10', 'V_1']
save_folders = ['REFL_10CM',  'OLR', 'Wind', 'Shear', 'CL_HT', 'TB', 'CAPE', 'RAIN', 'WIND_sfc', 'theta']
cmaps = ['turbo', 'binary', 'turbo', 'turbo', 'grey_to_white', 'binary', 'turbo', 'GnBu', 'turbo', 'turbo'] 
bottoms = [1, None, None, None, None, None, 1, .01, None, None]
tops = [None, None, None, 60, None, None, None, 10, None, None]

for var, title, x_barb, y_barb, save_folder, cmap, bottom, top in zip(vars_of_interest[-3:], titles[-3:], x_barbs[-3:], y_barbs[-3:], save_folders[-3:], cmaps[-3:], bottoms[-3:], tops[-3:]):
    print(var)
    make_maps(combined_dataset, var, title, x_barb, y_barb, bottom = bottom, top = top, cmap = cmap, save = True, save_loc = 'images/' + save_folder + '/')
    make_gif('images/'+ save_folder)




RAIN
1712
1713
1714
1715
1716
1717
1718
1719
1720
1721
1722
1723
1800
1801
1802
1803
1804
1805
1806
1807


C:\Users\miles\AppData\Local\Temp\ipykernel_49908\221254466.py:42: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure(figsize=(16, 12))


1808
1809
1810
1811
1812
WIND_sfc_mag
1712
1713
1714
1715
1716
1717
1718
1719
1720
1721
1722
1723


In [ ]:
combined_dataset.to_netcdf('excluded_data/dataset.nc')

In [3]:
xr.open_dataset('excluded_data/dataset.nc')

<xarray.Dataset>
Dimensions:                (time: 25, south_north: 282, west_east: 405,
                            bottom_top: 37, bottom_top_stag: 38,
                            soil_layers_stag: 4, west_east_stag: 406,
                            south_north_stag: 283, seed_dim_stag: 2)
Coordinates:
    XLAT                   (south_north, west_east) float32 ...
    XLONG                  (south_north, west_east) float32 ...
    XTIME                  (time) datetime64[ns] ...
    XLAT_U                 (south_north, west_east_stag) float32 ...
    XLONG_U                (south_north, west_east_stag) float32 ...
    XLAT_V                 (south_north_stag, west_east) float32 ...
    XLONG_V                (south_north_stag, west_east) float32 ...
Dimensions without coordinates: time, south_north, west_east, bottom_top,
                                bottom_top_stag, soil_layers_stag,
                                west_east_stag, south_north_stag, seed_dim_stag
Data variables: (12/149)
    Times                  (time) |S19 ...
    LU_INDEX               (time, south_north, west_east) float32 ...
    ZNU                    (time, bottom_top) float32 ...
    ZNW                    (time, bottom_top_stag) float32 ...
    ZS                     (time, soil_layers_stag) float32 ...
    DZS                    (time, soil_layers_stag) float32 ...
    ...                     ...
    CAPE                   (time, south_north, west_east) float32 ...
    U_shear_6              (time, south_north, west_east) float32 ...
    V_shear_6              (time, south_north, west_east) float32 ...
    mag_shear_6            (time, south_north, west_east) float32 ...
    TB                     (time, south_north, west_east) float32 ...
    CL_HT                  (time, south_north, west_east) float32 ...
Attributes: (12/146)
    TITLE:                            OUTPUT FROM WRF V4.1.3 MODEL
    START_DATE:                      2024-08-17_00:00:00
    SIMULATION_START_DATE:           2024-08-17_00:00:00
    WEST-EAST_GRID_DIMENSION:        406
    SOUTH-NORTH_GRID_DIMENSION:      283
    BOTTOM-TOP_GRID_DIMENSION:       38
    ...                              ...
    ISLAKE:                          -1
    ISICE:                           15
    ISURBAN:                         13
    ISOILWATER:                      14
    HYBRID_OPT:                      2
    ETAC:                            0.2